In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import random as rd
data = pd.read_csv("drive/My Drive/clean_tweet.csv",index_col=0)
data.head()

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,tweet_text,target
0,awww that bummer you shoulda got david carr of...,0
1,is upset that he can not update his facebook b...,0
2,dived many times for the ball managed to save ...,0
3,my whole body feels itchy and like its on fire,0
4,no it not behaving at all mad why am here beca...,0


In [3]:
data[data.isnull().any(axis=1)].head(20)


,tweet_text,target
208,NaN,0
249,NaN,0
282,NaN,0
398,NaN,0
430,NaN,0
1011,NaN,0
1014,NaN,0
1231,NaN,0
1421,NaN,0
1486,NaN,0


In [4]:
data.dropna(inplace=True) #NAN values are coming from just @username, delete them 
data.reset_index(drop=True,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596041 entries, 0 to 1596040
Data columns (total 2 columns):
tweet_text    1596041 non-null object
target        1596041 non-null int64
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [5]:
data[data.isnull().any(axis=1)].head() #No more null data


,tweet_text,target


In [0]:
x = data.tweet_text
y = data.target


In [0]:
# from sklearn.model_selection import train_test_split
# SEED = 2000
# x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
# x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [0]:
from tqdm import tqdm
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils

def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(t.split(), [prefix + '_%s' % i]))
    return result

In [0]:
all_x = pd.concat([x])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

In [28]:
cores = multiprocessing.cpu_count()
model_ug_cbow = Word2Vec(sg=0, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_cbow.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 1596041/1596041 [00:00<00:00, 3010277.03it/s]


In [29]:
%%time
for epoch in range(30):
    model_ug_cbow.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_cbow.alpha -= 0.002
    model_ug_cbow.min_alpha = model_ug_cbow.alpha

100%|██████████| 1596041/1596041 [00:00<00:00, 3225954.37it/s]


CPU times: user 20min 15s, sys: 3.96 s, total: 20min 19s
Wall time: 10min 56s


In [30]:
model_ug_sg = Word2Vec(sg=1, size=100, negative=5, window=2, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_sg.build_vocab([x.words for x in tqdm(all_x_w2v)])

100%|██████████| 1596041/1596041 [00:00<00:00, 3134491.94it/s]


In [31]:
%%time
for epoch in range(30):
    model_ug_sg.train(utils.shuffle([x.words for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_sg.alpha -= 0.002
    model_ug_sg.min_alpha = model_ug_sg.alpha

100%|██████████| 1596041/1596041 [00:00<00:00, 3036501.88it/s]


CPU times: user 35min 16s, sys: 4.05 s, total: 35min 20s
Wall time: 18min 25s


In [32]:
model_ug_cbow.save('drive/My Drive/w2v_model_ug_cbow.word2vec')
model_ug_sg.save('drive/My Drive/w2v_model_ug_sg.word2vec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [34]:
from gensim.models import KeyedVectors
model_ug_cbow = KeyedVectors.load('drive/My Drive/w2v_model_ug_cbow.word2vec')
model_ug_sg = KeyedVectors.load('drive/My Drive/w2v_model_ug_sg.word2vec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [35]:
len(model_ug_cbow.wv.vocab.keys())


106259

In [36]:
embeddings_index = {}
for w in model_ug_cbow.wv.vocab.keys():
    embeddings_index[w] = np.append(model_ug_cbow.wv[w],model_ug_sg.wv[w])
print('Found %s word vectors.' % len(embeddings_index))

Found 106259 word vectors.


In [37]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

Using TensorFlow backend.


In [38]:
len(tokenizer.word_index)


264939

In [39]:
x_train_seq = pad_sequences(sequences, maxlen=50)
print('Shape of data tensor:', x_train_seq.shape)

Shape of data tensor: (1596041, 50)


In [0]:
num_words = 100000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [41]:
np.array_equal(embedding_matrix[6] ,embeddings_index.get('you'))


True

In [43]:
y

0          0
1          0
2          0
3          0
4          0
          ..
1596036    4
1596037    4
1596038    4
1596039    4
1596040    4
Name: target, Length: 1596041, dtype: int64

In [0]:
y = np.array(list(map(lambda x: 0 if x==0 else 1, y))) 

In [46]:
len(y)

1596041

In [0]:
y_train = y

In [0]:
seed = 7
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D



In [0]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model

model_cnn_03 = Sequential()
e = Embedding(100000, 200, weights=[embedding_matrix], input_length=50, trainable=True)
model_cnn_03.add(e)
model_cnn_03.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
model_cnn_03.add(GlobalMaxPooling1D())
model_cnn_03.add(Dense(256, activation='relu'))
model_cnn_03.add(Dense(1, activation='sigmoid'))
model_cnn_03.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [82]:
from keras.callbacks import ModelCheckpoint
filepath="CNN_best_weights.{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

model_cnn_03.fit(x_train_seq, y_train, batch_size=32, epochs=5,
                     validation_split=0.01, callbacks = [checkpoint])

Train on 1580080 samples, validate on 15961 samples
Epoch 1/5
1580080/1580080 [==============================] - 515s 326us/step - loss: 0.4014 - acc: 0.8173 - val_loss: 0.4249 - val_acc: 0.8038

Epoch 00001: val_acc improved from -inf to 0.80383, saving model to CNN_best_weights.01-0.8038.hdf5
Epoch 2/5
1580080/1580080 [==============================] - 517s 327us/step - loss: 0.3647 - acc: 0.8380 - val_loss: 0.3764 - val_acc: 0.8239

Epoch 00002: val_acc improved from 0.80383 to 0.82388, saving model to CNN_best_weights.02-0.8239.hdf5
Epoch 3/5
1580080/1580080 [==============================] - 523s 331us/step - loss: 0.3405 - acc: 0.8514 - val_loss: 0.4059 - val_acc: 0.8097

Epoch 00003: val_acc did not improve from 0.82388
Epoch 4/5
1580080/1580080 [==============================] - 515s 326us/step - loss: 0.3175 - acc: 0.8633 - val_loss: 0.4108 - val_acc: 0.8176

Epoch 00004: val_acc did not improve from 0.82388
Epoch 5/5
1580080/1580080 [==============================] - 517s 327

In [83]:
columns = ['polarity','tweet_id','date','query_string','username','tweet_text']


test = pd.read_csv("drive/My Drive/testdata.csv", sep=",", header = None, names=columns)
test

,polarity,tweet_id,date,query_string,username,tweet_text
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,Reading my kindle2... Love it... Lee childs i...
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"Ok, first assesment of the #kindle2 ...it fuck..."
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,@kenburbary You'll love your Kindle2. I've had...
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,@mikefish Fair enough. But i have the Kindle2...
...,...,...,...,...,...,...
493,2,14072,Sun Jun 14 04:31:43 UTC 2009,latex,proggit,Ask Programming: LaTeX or InDesign?: submitted...
494,0,14073,Sun Jun 14 04:32:17 UTC 2009,latex,sam33r,"On that note, I hate Word. I hate Pages. I hat..."
495,4,14074,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...
496,0,14075,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."


In [84]:
test.polarity.value_counts() #80,000 positive and negative examples 


4    182
0    177
2    139
Name: polarity, dtype: int64

In [0]:
test = test[test.polarity != 2] #Drop neutral rows for binary classification (postive or negative)


In [86]:
x_test = test['tweet_text']
x_test

0      @stellargirl I loooooooovvvvvveee my Kindle2. ...
1      Reading my kindle2...  Love it... Lee childs i...
2      Ok, first assesment of the #kindle2 ...it fuck...
3      @kenburbary You'll love your Kindle2. I've had...
4      @mikefish  Fair enough. But i have the Kindle2...
                             ...                        
492    After using LaTeX a lot, any other typeset mat...
494    On that note, I hate Word. I hate Pages. I hat...
495    Ahhh... back in a *real* text editing environm...
496    Trouble in Iran, I see. Hmm. Iran. Iran so far...
497    Reading the tweets coming out of Iran... The w...
Name: tweet_text, Length: 359, dtype: object

In [87]:
y_test = test['polarity']
y_test

0      4
1      4
2      4
3      4
4      4
      ..
492    4
494    0
495    4
496    0
497    0
Name: polarity, Length: 359, dtype: int64

In [0]:
X_test = []
tweets = list(test['tweet_text'])
for lines in tweets:
    X_test.append(lines)

y_test = test['polarity']

#1 == positive, 0 == negative
y_test = np.array(list(map(lambda x: 0 if x==0 else 1, y_test))) 

In [0]:
sequences_test = tokenizer.texts_to_sequences(X_test)
x_test_seq = pad_sequences(sequences_test, maxlen=50)

In [90]:
# score = loaded_CNN_model.evaluate(x=X_test, y=y_test)
from keras.models import load_model
loaded_CNN_model = load_model('CNN_best_weights.02-0.8239.hdf5')
score = loaded_CNN_model.evaluate(x=x_test_seq, y=y_test)

359/359 [==============================] - 0s 599us/step


In [91]:
score[0]

0.4112673430057621

In [92]:
score[1]

0.816155989189998